### 第3章: 正規表現
Wikipediaの記事を以下のフォーマットで書き出したファイルjawiki-country.json.gzがある．

1行に1記事の情報がJSON形式で格納される 各行には記事名が”title”キーに，記事本文が”text”キーの辞書オブジェクトに格納され，そのオブジェクトがJSON形式で書き出される ファイル全体はgzipで圧縮される 以下の処理を行うプログラムを作成せよ．

In [1]:
!mkdir -p data/chapter3 && wget -P data/chapter3/ https://nlp100.github.io/data/jawiki-country.json.gz
!gunzip data/chapter3/jawiki-country.json.gz

--2023-11-23 16:00:01--  https://nlp100.github.io/data/jawiki-country.json.gz
Resolving nlp100.github.io (nlp100.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5068362 (4.8M) [application/gzip]
Saving to: ‘data/chapter3/jawiki-country.json.gz’

jawiki-country.json 100%[===================>]   4.83M  24.7MB/s    in 0.2s    

2023-11-23 16:00:01 (24.7 MB/s) - ‘data/chapter3/jawiki-country.json.gz’ saved [5068362/5068362]



### 20. JSONデータの読み込み
Wikipedia記事のJSONファイルを読み込み，「イギリス」に関する記事本文を表示せよ．問題21-29では，ここで抽出した記事本文に対して実行せよ．

In [2]:
!cat data/chapter3/jawiki-country.json | jq 'select(.title=="イギリス")'  | jq '.text' | sed -e 's/\\n/\n/g' > data/chapter3/20.out

### 21. カテゴリ名を含む行を抽出
記事中でカテゴリ名を宣言している行を抽出せよ．

In [11]:
!cat data/chapter3/20.out | grep '\[Category'

[[Category:イギリス|*]]
[[Category:イギリス連邦加盟国]]
[[Category:英連邦王国|*]]
[[Category:G8加盟国]]
[[Category:欧州連合加盟国|元]]
[[Category:海洋国家]]
[[Category:現存する君主国]]
[[Category:島国]]
[[Category:1801年に成立した国家・領域]]"


### 22. カテゴリ名の抽出
記事のカテゴリ名を（行単位ではなく名前で）抽出せよ．

In [12]:
!cat data/chapter3/20.out | grep '\[\[Category' | sed -E 's/\[\[Category:(.*)]].?$/\1/' | sed -e 's/[\|\*]//g'

イギリス
イギリス連邦加盟国
英連邦王国
G8加盟国
欧州連合加盟国元
海洋国家
現存する君主国
島国
1801年に成立した国家・領域


### 23. セクション構造
記事中に含まれるセクション名とそのレベル（例えば”== セクション名 ==”なら1）を表示せよ．

In [50]:
import re

with open('data/chapter3/20.out', 'r') as f:
    string = ''.join(f.readlines())
    pattern = r'(\={2,})(.*?)\={2,}'
    n_labels = re.findall(pattern, string)
    result = '\n'.join(["{0}:{1}".format(x[1], len(x[0])-1) for x in n_labels])
    print(result)


国名:1
歴史:1
地理:1
主要都市:2
気候:2
政治:1
元首:2
法:2
内政:2
地方行政区分:2
外交・軍事:2
経済:1
鉱業:2
農業:2
貿易:2
不動産:2
エネルギー政策:2
通貨:2
企業:2
通信:3
交通:1
道路:2
鉄道:2
海運:2
航空:2
科学技術:1
国民:1
言語:2
宗教:2
婚姻:2
移住:2
教育:2
医療:2
文化:1
食文化:2
文学:2
哲学:2
音楽:2
ポピュラー音楽:3
映画:2
コメディ:2
国花:2
世界遺産:2
祝祭日:2
スポーツ:2
サッカー:3
クリケット:3
競馬:3
モータースポーツ:3
野球:3
 カーリング :3
 自転車競技 :3
脚注:1
関連項目:1
外部リンク:1


### 24. ファイル参照の抽出
記事から参照されているメディアファイルをすべて抜き出せ．

In [8]:
import re

with open('data/chapter3/20.out', 'r') as f:
    string = ''.join(f.readlines())
    pattern = r'\[\[ファイル:(.*?)\|'
    res = '\n'.join(re.findall(pattern, string))
    print(res)


Royal Coat of Arms of the United Kingdom.svg
Descriptio Prime Tabulae Europae.jpg
Lenepveu, Jeanne d'Arc au siège d'Orléans.jpg
London.bankofengland.arp.jpg
Battle of Waterloo 1815.PNG
Uk topo en.jpg
BenNevis2005.jpg
Population density UK 2011 census.png
2019 Greenwich Peninsula & Canary Wharf.jpg
Birmingham Skyline from Edgbaston Cricket Ground crop.jpg
Leeds CBD at night.jpg
Glasgow and the Clyde from the air (geograph 4665720).jpg
Palace of Westminster, London - Feb 2007.jpg
Scotland Parliament Holyrood.jpg
Donald Trump and Theresa May (33998675310) (cropped).jpg
Soldiers Trooping the Colour, 16th June 2007.jpg
City of London skyline from London City Hall - Oct 2008.jpg
Oil platform in the North SeaPros.jpg
Eurostar at St Pancras Jan 2008.jpg
Heathrow Terminal 5C Iwelumo-1.jpg
Airbus A380-841 G-XLEB British Airways (10424102995).jpg
UKpop.svg
Anglospeak.svg
Royal Aberdeen Children's Hospital.jpg
CHANDOS3.jpg
The Fabs.JPG
Wembley Stadium, illuminated.jpg


### 25. テンプレートの抽出
記事中に含まれる「基礎情報」テンプレートのフィールド名と値を抽出し，辞書オブジェクトとして格納せよ．

In [163]:
import re

with open('data/chapter3/20.out', 'r') as f:
    string = ''.join(f.readlines())
    pattern = r'^\{\{基礎情報 国(.*)^\}\}'
    matched = ''.join(re.findall(pattern, string, re.MULTILINE + re.DOTALL))\
                .translate(str.maketrans({' ': '', '　': ''}))\
    
    # (?:(?=\n\|)|(?=\n)$)
    # ?: 非キャプチャグループ。後続のOR（|）をグルーピングさせたいが、キャプチャはさせる必要はない。
    # (?=\n\|) 最終行以外は '\n|' を終端とする。ただし肯定先読み'?='を利用してそれ自体は次以降のマッチングにも利用する
    # (?=\n)$) 最終行のマッチング用
    pattern = r'^\|(.*?=.*?)(?:(?=\n\|)|(?=\n)$)'
    matched = re.findall(pattern, matched, re.MULTILINE + re.DOTALL)

    res = {}
    for s in matched:
        splited = s.split('=')
        res[splited[0]] = splited[1]
        print(f'{splited[0]}:{splited[1]}')
    


略名:イギリス
日本語国名:グレートブリテン及び北アイルランド連合王国
公式国名:{{lang|en|UnitedKingdomofGreatBritainandNorthernIreland}}<ref>英語以外での正式国名:<br/>
国旗画像:FlagoftheUnitedKingdom.svg
国章画像:[[ファイル:RoyalCoatofArmsoftheUnitedKingdom.svg|85px|イギリスの国章]]
国章リンク:（[[イギリスの国章|国章]]）
標語:{{lang|fr|[[Dieuetmondroit]]}}<br/>（[[フランス語]]:[[Dieuetmondroit|神と我が権利]]）
国歌:[[女王陛下万歳|{{lang|en|GodSavetheQueen}}]]{{enicon}}<br/>''神よ女王を護り賜え''<br/>{{center|[[ファイル:UnitedStatesNavyBand-GodSavetheQueen.ogg]]}}
地図画像:Europe-UK.svg
位置画像:UnitedKingdom(+overseasterritories)intheWorld(+Antarcticaclaims).svg
公用語:[[英語]]
首都:[[ロンドン]]（事実上）
最大都市:ロンドン
元首等肩書:[[イギリスの君主|女王]]
元首等氏名:[[エリザベス2世]]
首相等肩書:[[イギリスの首相|首相]]
首相等氏名:[[ボリス・ジョンソン]]
他元首等肩書1:[[貴族院(イギリス)|貴族院議長]]
他元首等氏名1:[[:en:NormanFowler,BaronFowler|ノーマン・ファウラー]]
他元首等肩書2:[[庶民院(イギリス)|庶民院議長]]
他元首等氏名2:{{仮リンク|リンゼイ・ホイル|en|LindsayHoyle}}
他元首等肩書3:[[連合王国最高裁判所|最高裁判所長官]]
他元首等氏名3:[[:en:BrendaHale,BaronessHaleofRichmond|ブレンダ・ヘイル]]
面積順位:76
面積大きさ:1E11
面積値:244,820
水面積率:1.3%
人口統計年:2018
人口順位:22
人口大きさ:1E7
人口値:6643万5600<ref>{{Citeweb|u

### 26. 強調マークアップの除去
25の処理時に，テンプレートの値からMediaWikiの強調マークアップ（弱い強調，強調，強い強調のすべて）を除去してテキストに変換せよ．

In [169]:
import re

# 要: Q25 の res 変数の読み込み
for k, v in res.items():
    v = re.sub(r'\'{2,5}', '', v)
    print(f'{k}:{v}')


略名:イギリス
日本語国名:グレートブリテン及び北アイルランド連合王国
公式国名:{{lang|en|UnitedKingdomofGreatBritainandNorthernIreland}}<ref>英語以外での正式国名:<br/>
国旗画像:FlagoftheUnitedKingdom.svg
国章画像:[[ファイル:RoyalCoatofArmsoftheUnitedKingdom.svg|85px|イギリスの国章]]
国章リンク:（[[イギリスの国章|国章]]）
標語:{{lang|fr|[[Dieuetmondroit]]}}<br/>（[[フランス語]]:[[Dieuetmondroit|神と我が権利]]）
国歌:[[女王陛下万歳|{{lang|en|GodSavetheQueen}}]]{{enicon}}<br/>神よ女王を護り賜え<br/>{{center|[[ファイル:UnitedStatesNavyBand-GodSavetheQueen.ogg]]}}
地図画像:Europe-UK.svg
位置画像:UnitedKingdom(+overseasterritories)intheWorld(+Antarcticaclaims).svg
公用語:[[英語]]
首都:[[ロンドン]]（事実上）
最大都市:ロンドン
元首等肩書:[[イギリスの君主|女王]]
元首等氏名:[[エリザベス2世]]
首相等肩書:[[イギリスの首相|首相]]
首相等氏名:[[ボリス・ジョンソン]]
他元首等肩書1:[[貴族院(イギリス)|貴族院議長]]
他元首等氏名1:[[:en:NormanFowler,BaronFowler|ノーマン・ファウラー]]
他元首等肩書2:[[庶民院(イギリス)|庶民院議長]]
他元首等氏名2:{{仮リンク|リンゼイ・ホイル|en|LindsayHoyle}}
他元首等肩書3:[[連合王国最高裁判所|最高裁判所長官]]
他元首等氏名3:[[:en:BrendaHale,BaronessHaleofRichmond|ブレンダ・ヘイル]]
面積順位:76
面積大きさ:1E11
面積値:244,820
水面積率:1.3%
人口統計年:2018
人口順位:22
人口大きさ:1E7
人口値:6643万5600<ref>{{Citeweb|url
人

### 27. 内部リンクの除去
26の処理に加えて，テンプレートの値からMediaWikiの内部リンクマークアップを除去し，テキストに変換せよ．